In [0]:
from zipfile import ZipFile
file_name = "training.5k.zip"

with ZipFile(file_name, 'r') as zip:
  zip.extractall()
  print('done')

In [0]:
import pickle

pickle_out = open("train_data_240*240.pickle","wb")
pickle.dump(train_data, pickle_out)
pickle_out.close()

pickle_out = open("labels_240*240.pickle","wb")
pickle.dump(labels, pickle_out)
pickle_out.close()

In [0]:
IMG_SIZE=240
LR = 1e-3
MODEL_NAME='PSSHRI'
TRAIN_DIR="training/training"

In [0]:
solution = pd.read_csv("training/solution.csv")

In [0]:
train_data=[]
labels=[]
for img in tqdm(os.listdir(TRAIN_DIR)):
    path = os.path.join(TRAIN_DIR,img)
    img_data = cv2.imread(path)
    img_data = cv2.resize(img_data, (IMG_SIZE,IMG_SIZE))
    train_data.append([np.array(img_data)])
    number = int(img[:-4])
    categ_num = int(solution[solution.id==number].category)
    labels.append(categ_num)

In [0]:
lb=LabelBinarizer()
labels=lb.fit_transform(labels)

In [0]:
pickle_in = open("train_data.pickle","rb")
train_data = pickle.load(pickle_in)

pickle_in = open("labels.pickle","rb")
labels = pickle.load(pickle_in)

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
import natsort
import keras
from tqdm import tqdm
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Dense, Flatten, Dropout
from keras.layers.normalization import BatchNormalization
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import ImageDataGenerator
from sklearn import model_selection
import pickle
from keras.models import load_model
from sklearn.preprocessing import LabelBinarizer
from keras.callbacks import ModelCheckpoint
from keras.layers.advanced_activations import LeakyReLU

In [0]:
#train_data=np.array(train_data,dtype="float32")/255.0
#labels=np.array(labels)

In [0]:
#lb=LabelBinarizer()
#labels=lb.fit_transform(labels)
x_train,x_test,y_train,y_test=model_selection.train_test_split(train_data,labels,test_size=0.25,random_state=42)

In [0]:
aug=ImageDataGenerator(rotation_range=25,width_shift_range=0.1,height_shift_range=0.1,shear_range=0.2,horizontal_flip=True,fill_mode="nearest")

In [0]:
model=Sequential()
model.add(Conv2D(32,(3,3),padding="same",activation="linear",input_shape=(240,240,3)))
model.add(LeakyReLU(alpha=0.1))
model.add(BatchNormalization(axis=-1))
model.add(MaxPooling2D(pool_size=(3,3)))
model.add(Dropout(0.25))

model.add(Conv2D(32,(3,3),padding="same",activation="linear"))
model.add(LeakyReLU(alpha=0.1))
model.add(BatchNormalization(axis=-1))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

model.add(Conv2D(64,(3,3),activation='relu'))
model.add(Conv2D(64,(3,3),padding="same",activation="linear"))
model.add(LeakyReLU(alpha=0.1))
model.add(Dropout(0.25))

model.add(Conv2D(128,(3,3),padding="same",activation="linear"))
model.add(LeakyReLU(alpha=0.1))
model.add(BatchNormalization(axis=-1))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(128,(3,3),padding="same",activation="relu"))
model.add(BatchNormalization(axis=-1))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(1024,activation="linear"))
model.add(LeakyReLU(alpha=0.1))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(6,activation="softmax"))

In [24]:
model.compile(loss=keras.losses.categorical_crossentropy,optimizer=keras.optimizers.Adam(),metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_7 (Conv2D)            (None, 240, 240, 32)      896       
_________________________________________________________________
leaky_re_lu_6 (LeakyReLU)    (None, 240, 240, 32)      0         
_________________________________________________________________
batch_normalization_6 (Batch (None, 240, 240, 32)      128       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 80, 80, 32)        0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 80, 80, 32)        0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 80, 80, 32)        9248      
_________________________________________________________________
leaky_re_lu_7 (LeakyReLU)    (None, 80, 80, 32)        0         
__________

In [25]:
train=model.fit_generator(aug.flow(x_train,y_train,batch_size=64),validation_data=(x_test,y_test),steps_per_epoch=len(x_train)/64,epochs=2,verbose=1)

ValueError: ignored